In [ ]:
using BandedMatrices, LinearAlgebra, SparseArrays, IntervalArithmetic, Serialization, SpecialFunctions, Arblib

In [ ]:
Base.:*(A::SparseMatrixCSC{Interval{Float64}, Int64},x::Vector{Interval{Float64}}) = Vector((A*sparse(x[:,:]))[:])
Base.:*(A::SparseMatrixCSC{Interval{Float64}, Int64}, B::Adjoint) = A*Matrix(B)
Arb2Int(x) = interval(Float64(getinterval(x)[1], RoundDown), Float64(getinterval(x)[2], RoundUp))
Int2Arb(x) = Arb((inf(x), sup(x)))
SpecialFunctions.gamma(x::Interval{Float64}) = Arb2Int(gamma(Int2Arb(x)))

In [ ]:
β̲ = -interval(Float64, 11//100)
β̄ = interval(Float64, 0)
B = interval(Float64, 124//100)

K = 8
βK = (β̲ + β̄)/interval(2) .+cos.(interval.(Float64, collect(K:-1:0).//(K))*interval(π))*(β̄ - β̲)/interval(2)
θK = interval.(Float64, collect(K:-1:0).//(K))*interval(π)
xK = cos.(θK)
indK = interval.(collect(0:K))
# MK maps a K+1 Chebyshev coefficients to evaluation at K+1 Chebyshev nodes
MK = cos.(θK*indK')
MK[:,2:end] *=interval(2)
# MKinv maps the evalation at Chebyshev at K+1 nodes to K+1 Chebyshev coefficients
MKinv = (cos.(θK*indK')/interval(K))'
MKinv[:,1]/=interval(2)
MKinv[:,end]/=interval(2)
MKinv[end,:]/=interval(2);

βK₁ = (β̲ + β̄)/interval(2) .+cos.(interval.(Float64, collect(K+1:-1:0).//(K+1))*interval(π))*(β̄ - β̲)/interval(2)
θK₁ = interval.(Float64, collect(K+1:-1:0).//(K+1))*interval(π)
xK₁ = cos.(θK₁)
indK₁ = interval.(collect(0:K+1))
# MK maps a K+2 Chebyshev coefficients to evaluation at K+2 Chebyshev nodes
MK₁ = cos.(θK₁*indK₁')
MK₁[:,2:end] *=interval(2)
# MKinv maps the evalation at Chebyshev at K+2 nodes to K+2 Chebyshev coefficients
MK₁inv = (cos.(θK₁*indK₁')/interval(K+1))'
MK₁inv[:,1]/=interval(2)
MK₁inv[:,end]/=interval(2)
MK₁inv[end,:]/=interval(2);

In [ ]:
function D(N)
    # implements ∂ₓ in Fourier
    v = interval.([ ((n+1 ) % 2) * (n÷2) for n=1:2*N])
    return dropzeros(sparse(BandedMatrix(-1 => v, 1 =>-v)))
end

function D2(N)
    # implements ∂ₓₓ in Fourier
    v = interval.([-(n÷2)^2 for n=1:2*N+1])
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 2)))[1:2*N+1,1:2*N+1]))
end

function J(N)
    # implements ∂ₓₓ in Fourier
    v = interval(1)./interval.([1+(n÷2)^2 for n=1:2*N+1])
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 2)))[1:2*N+1,1:2*N+1]))
end

function id(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => v)))
end

function id2(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 2)))[:,1:2*N+1]))
end

function idZ(N)
    v = ones(Interval{Float64}, N+1)
    return dropzeros(sparse(BandedMatrix( 0 => v)))
end

function idZ2(N)
    v = ones(Interval{Float64}, N+2)
    return dropzeros(sparse(BandedMatrix( 0 => v)))[:,1:N+1]
end

function C(N)
    # implements u ↦ cosx u in Fourier
    v = vcat([interval(0.0)], ones(Interval{Float64}, 2*N)/interval(2))
    A = dropzeros(sparse(BandedMatrix( -2 => v, 2 => v[1:end-2])[:,1:2*N+1]))
    A[1,3] = interval(0.5)
    A[3,1] = interval(1.0)
    return A
end

function S(N)
    # implements u ↦ sinx u in Fourier
    v = vcat([interval(0.0)],interval.([ ((n+1 ) % 2) for n=1:2*N]))/interval(2)
    A = dropzeros(sparse(BandedMatrix( -1 => v[1:end], -3 =>-v[2:end], 1 => v, 3=>-v[2:end])[:,1:2*N+1]))
    A[1,2] = interval(0.5)
    A[2,1] = interval(1.0)
    return A
end

function L(N)
    u = interval.(1:N+1).^2
    v = -interval(0:N+1).*interval(2*(0:N+1).+1)
    w = interval(0:N).*interval((0:N).+1)
    return dropzeros(sparse(BandedMatrix(1 => u, 0 => v, -1=> w)[:,1:N+1]))
end

function Z(N)
    u = -interval.(1:N+1)
    v = interval(2*(0:N+1).+1)
    w = -interval((0:N).+1)
    return dropzeros(sparse(Matrix(BandedMatrix(1 => u, 0 => v, -1=> w)[:,1:N+1])))
end

function ZD(N)
    u = -interval.(1:N+1)
    v = interval.(0:N+1)
    return dropzeros(sparse(BandedMatrix(1 => u, 0 => v)[:,1:N+1]))
end

In [ ]:
N = 20000
M = 200;

In [ ]:
# preconditioning matrix
A = kron(sparse(I(2*M+3)), sparse(Diagonal(1 ./(1:N+2).^1.0)));

In [ ]:
𝔄 = kron(id2(M),L(N)/interval(4))
𝔄 += kron(id2(M), interval(1//4)*ZD(N))
𝔄 += kron((interval(3)*id2(M)+interval(3)*C(M)+B*S(M))*D(M), Z(N)/(interval(4)*B));
𝔄 += kron((id2(M)-C(M))*D2(M), idZ2(N)/interval(2))
𝔄 -= kron(S(M)*D(M), ZD(N)/interval(2))
𝔄 = [kron(id2(M),(idZ2(N)-Z(N))/interval(8))[:,1] 𝔄[:,2:end]];

In [ ]:
𝔅 = kron(id2(M), ZD(N))
𝔅 = [kron(id2(M),idZ2(N)/interval(2))[:,1] 𝔅[:,2:end]];

In [ ]:
Q = -Vector(kron(interval(2)*id2(M)+C(M), Z(N))[:,1]/interval(8))
Q += Vector(kron(S(M), Z(N))[:,1]*interval(Float64, 3//8)/B);
e₁ = zeros(Interval{Float64}, size(Q))
e₁[1] = interval(1)
Q += e₁/interval(8);
QK = Q' .+ βK*e₁'/interval(2)
QK₁ = Q' .+ βK₁*e₁'/interval(2);

In [ ]:
ūK = zeros(Float64,(K+1,size(𝔄)[2]));

In [ ]:
for i=1:K+1
# for i=8:8
    println(i)
    β = βK[i]
    ūK[i,:] = interval.((A*(mid.(𝔄)+mid(β)*mid.(𝔅)))[2:end,:] \ (A*collect(mid.(QK[i,:])))[2:end]);
end

In [ ]:
# serialize("uK3", ūK)
# ūK = deserialize("uK3")
ūK = interval.(ūK);

In [ ]:
GC.gc()

In [ ]:
ū_coeffs = zeros(Interval{Float64}, size(ūK))
# currently faster than matmul
Threads.@threads for i=1:size(ūK)[2]
    ū_coeffs[:,i] = MKinv*ūK[:,i]
end
ū_coeffs = vcat(ū_coeffs, zeros(Interval{Float64},(1, size(𝔄)[2])))
ūK₁ = zeros(Interval{Float64}, size(ū_coeffs))
Threads.@threads for i=1:size(ū_coeffs)[2]
    ūK₁[:,i] = MK₁*ū_coeffs[:,i]
end

In [ ]:
ϵK₁ = Matrix(reduce(hcat,[((𝔄*ūK₁[i,:]) + βK₁[i]*(𝔅*ūK₁[i,:])) - QK₁[i,:] for i=1:K+2])');

In [ ]:
λ̄K₁ = -ϵK₁[:,1]
ϵK₁[:,1] .= interval(0);

In [ ]:
ϵ_coeffs = zeros(Interval{Float64}, size(ϵK₁))
Threads.@threads for i = 1:size(ϵK₁)[2]
    # println(i)
    ϵ_coeffs[:,i] = MK₁inv*ϵK₁[:,i]
end
ϵ_coeffs[2:end,:] *= interval(2);

In [ ]:
ϵ_sups = sum(ϵ_coeffs, dims = 1)
ϵ_sups = reshape(ϵ_sups, (N+2, 2*M+3));

In [ ]:
βs = mince(interval(β̲, β̄), 100000);
c = maximum(sqrt.(gamma.(interval(1) .+ interval(8)*βs))./gamma.(interval(1) .+ interval(4)*βs))

In [ ]:
δ = sum([sqrt(sum(ϵ_sups[:,i].^2)) for i=1:2*M+3])*c

In [ ]:
λ̄_coeffs = MK₁inv*λ̄K₁;
λ̄_coeffs[2:end] *= interval(2);

In [ ]:
function λ̄(β)
    return sum(λ̄_coeffs.*cos.(interval(0:K+1)*acos((β-(β̄+β̲)/interval(2))/(β̄-β̲)*interval(2))))
end

In [ ]:
x = mince(interval(0,π),100000)

if all(inf.(sum(λ̄_coeffs.*cos.(interval(0:K+1)*x'), dims = 1).-δ).>0)
    println("sign of Lyapunov exponents checked")
else
    println("sign of Lyapunov exponents NOT checked")
end

In [ ]:
β = interval.(collect(0:100)/100)*(β̄-β̲).+β̲;

In [ ]:
using Plots
plot(mid.(β), mid.(λ̄.(β)))
scatter!(mid.(βK₁), mid.(λ̄K₁))